<a href="https://colab.research.google.com/github/miguelamda/gtc2017-numba/blob/master/1%20-%20Numba%20Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Nociones Básicas de Numba

![logo](http://numba.pydata.org/_static/numba-blue-horizontal-rgb.svg)

## 1.1 ¿Qué es Numba?

[Numba](http://numba.pydata.org/) es un **function compiler**, **just-in-time**, y **type-specializing** para acelerar Python **enfocado numéricamente**. Esto es una lista larga, rompamos esto en los siguientes términos:

 * **function compiler**: Numba compila funciones de Python, no aplicaciones enteras ni partes de funciones.  Numba no reemplaza tu intérprete de Python, es tan solo otro módulo de Python que puede convertir tu función en otra (usualmente) más rápida. 
 * **type-specializing**: Numba acelera tu función generando una implementación especializada para los tipos de datos específicos que está utilizando.  Las funciones de Python están diseñadas para funcionar con tipos de datos genéricos, lo que las hace muy flexibles, pero también muy lentas.  En la práctica, uno solo llama a una función con un pequeño número de tipos de argumentos, por lo que Numba generará una implementación rápida para cada conjunto de tipos.
 * **just-in-time**: Numba traduce las funciones cuando se llaman por primera vez.  Esto asegura que el compilador sepa qué tipos de argumentos va a usar.  Esto también permite que Numba sea usado interactivamente en un cuaderno Jupyter tan fácilmente como una aplicación tradicional.
 * **numerically-focused**: Actualmente, Numba se centra en los tipos de datos numéricos, como `int`, `float`, y `complex`.  El soporte para el procesamiento de cadenas es muy limitado, y en muchos casos el uso de cadenas no funciona bien en la GPU.  Para obtener los mejores resultados con Numba, es probable que utilices los arrays de NumPy.

## 1.2 Requisitos

Numba en soportado en un gran número de sistemas operativos:

 * Windows 7 en adelante, 32 y 64-bit
 * macOS 10.9 en adelante, 64-bit
 * Linux (la mayoría >= RHEL 5), 32-bit y 64-bit

también en varias versiones de Python:

 * Python 2.7 y 3.6 en adelante
 * NumPy 1.15 en adelante

y en un gran variedad de hardware:

* CPUs x86, x86_64/AMD64
* CPUs ARM (ARMv7 en Raspberry Pi, ARMv8 en NVIDIA Jetson)
* GPUs de NVIDIA con CUDA (mínimo compute capability 3.0 y CUDA 7.5)
* GPUs de AMD con ROCm (solo linux y no en AMD Carrizo ni Kaveri APU)

En este tutorial, vamos a usar Linux 64-bit y CUDA 8.

## 1.3 Instalación

Se puede instalar tanto con Anaconda como con pip. Accede a la [web oficial](http://numba.pydata.org/) para ver el comando de instalación.

``` python
# Comando con pip
pip install numba
# Comando con Anacoda
conda install numba
```

## 1.4 Primeros Pasos

Escribamos nuestra primera función Numba y compilémosla para la **CPU**. El compilador de Numba se abilita típicamente aplicando un *decorador* a la función Python. Los decoradores son funciones que transforman funciones de Python. Aquí usamos el decorador para compilación CPU:

In [0]:
from numba import jit
import math

@jit
def hypot(x, y):
    # Implementación de https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

El código anterior es equivalente a escribir:

``` python
def hypot(x, y):
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)
    
hypot = jit(hypot)
```

Esto significa que el compilador de Numba es tan solo una función que puedes llamar siempre que quieras. Probemos nuestro cálculo de la hipotenusa:

In [2]:
hypot(3.0, 4.0)

5.0

La primera vez que llamamos a `hypot`, el compilador se lanza y compila una implementación de código máquina para las entradas de tipo float. Numba también salva la función Python original en el atributo `.py_func`, así que podemos llamar la código Python original para asegurarnos que obtenemos la misma respuesta:

In [3]:
hypot.py_func(3.0, 4.0)

5.0

## 1.5 Benchmarking

Una parte importante de usar Numba es la medición de rendimiento de nuestro nuevo código. Veamos si de verdad obtenemos algo de aceleración. La forma más sencilla de hacer esto en Jupyter notebook es usar la función magic `%timeit`. Primero midamos la velocidad del código Python original:

In [4]:
%timeit hypot.py_func(3.0, 4.0)

The slowest run took 15.83 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 633 ns per loop


El magic `%timeit` ejecuta la sentencia varias veces para obtener una estimación precisa del tiempo de ejecución. También retorna el mejor tiempo de ejecución por defecto, lo cual es útil para reducir la probabilidad de que eventos aleatorios en segundo plano afecten la medición. La mejor de las 3 aproximaciones también asegura que el tiempo de compilación de la primera llamada no rompa los resultados:

In [7]:
%timeit hypot(3.0, 4.0)

The slowest run took 24.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 201 ns per loop


Numba ha hecho un buen trabajo con esta función. Ahora es 4x (4 veces) más rápido que la versión pura de Python. 

Por supuesto, la función `hypot` también está presente en el módulo Python:

In [6]:
%timeit math.hypot(3.0, 4.0)

The slowest run took 129.83 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 110 ns per loop


¡El built-in de Python es incluso más rápido que el de Numba!  Esto se debe a que Numba introduce una sobrecarga en cada llamada a función que es más grande que la sobrecarga del propio Python.  Las funciones extremadamente rápidas (como la anterior) se verán perjudicadas por esto.

Sin embargo, si llamas a una función Numba desde otra, hay muy poca sobrecarga de función, a veces incluso cero si el compilador incrustra la función en la otra.

## 1.6 ¿Cómo Funciona Numba?

La primera vez que llamamos a nuestra función `hypot` envuelta en Numba, se inició el siguiente proceso:

![Numba Flowchart](img/numba_flowchart.png "The compilation process")

Podemos ver el resultado de la inferencia de tipos utilizando el método `.inspect_types()`, que imprime una versión anotada del código fuente:

In [8]:
hypot.inspect_types()

hypot (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-1-fad9c051470c>
# --- LINE 4 --- 

@jit

# --- LINE 5 --- 

def hypot(x, y):

    # --- LINE 6 --- 

    # Implementación de https://en.wikipedia.org/wiki/Hypot

    # --- LINE 7 --- 
    # label 0
    #   x = arg(0, name=x)  :: float64
    #   y = arg(1, name=y)  :: float64
    #   $0.1 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $0.3 = call $0.1(x, func=$0.1, args=[Var(x, <ipython-input-1-fad9c051470c>:7)], kws=(), vararg=None)  :: (float64,) -> float64
    #   del x
    #   del $0.1
    #   x.1 = $0.3  :: float64
    #   del $0.3

    x = abs(x);

    # --- LINE 8 --- 
    #   $0.4 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $0.6 = call $0.4(y, func=$0.4, args=[Var(y, <ipython-input-1-fad9c051470c>:7)], kws=(), vararg=None)  :: (float64,) -> float64
    #   del y
    #   del 

Nótese que los nombres de los tipos de Numba tienden a reflejar los nombres de los tipos de NumPy, así que un `float` de Python es un `float64` (también llamado "doble precisión" en otros idiomas).  Echar un vistazo a los tipos de datos a veces puede ser importante en el código GPU, porque el rendimiento de los cálculos con `float32` y `float64` será muy diferente en los dispositivos CUDA. Un casting de tipos accidental puede ralentizar muchísimo una función.

## 1.7 Cuando Las Cosas Van Mal

Numba no puede compilar todo el código Python.  Algunas funciones no tienen una traducción de Numba, y algunas clases de tipos Python no pueden ser compilados eficientemente (todavía).  Por ejemplo, Numba no soporta diccionarios (a día de hoy...):

In [9]:
@jit
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

<ipython-input-9-9ab63214e97a>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cannot_compile" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-9-9ab63214e97a> (3)

File "<ipython-input-9-9ab63214e97a>", line 3:
def cannot_compile(x):
    return x['key']
    ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "cannot_compile" was compiled in object mode without forceobj=True.

File "<ipython-input-9-9ab63214e97a>", line 2:
@jit
def cannot_compile(x):
^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall

'value'

Espera, ¿qué pasó?  Por defecto, Numba volverá a un modo, llamado "modo objeto" (object mode), que no hace especialización de tipos.  El modo objeto existe para habilitar otras funcionalidades de Numba, pero en muchos casos, es deseable que Numba te diga si la inferencia de tipos falla.  Puedes forzar el "modo nopython" ("nopython mode", que es el otro modo de compilación) pasando argumentos al decorador:

In [10]:
@jit(nopython=True)
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

TypingError: ignored

Ahora obtenemos una excepción cuando Numba intenta compilar la función, con un error que dice:

```
- argument 0: cannot determine Numba type of <class 'dict'>
```

que es el problema subyacente.

Veremos otros argumentos del decorador `@jit` en futuras secciones.


# II. Ejercicio Propuesto

A continuación se muestra una función que hace un bucle sobre dos matrices NumPy de entrada y pone su suma en la matriz de salida.  Modifica esta función para llamar a la función `hypot` que definimos arriba.  Aprenderemos una forma más eficiente de escribir tales funciones en una sección futura.

(Asegúrate de ejecutar todas las celdas de este cuaderno para que se defina `hypot`).

In [0]:
@jit(nopython=True)
def ex1(x, y, out):
    for i in range(x.shape[0]):
        out[i] = x[i] + y[i]

In [12]:
import numpy as np

in1 = np.arange(10, dtype=np.float64)
in2 = 2 * in1 + 1
out = np.empty_like(in1)

print('in1:', in1)
print('in2:', in2)

ex1(in1, in2, out)

print('out:', out)

in1: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
in2: [ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.]
out: [ 1.  4.  7. 10. 13. 16. 19. 22. 25. 28.]


In [13]:
# This test will fail until you fix the ex1 function
np.testing.assert_almost_equal(out, np.hypot(in1, in2))

AssertionError: ignored